<a href="https://www.kaggle.com/code/aisuko/goal-directed-dialog-godel?scriptVersionId=173110725" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview



In [1]:
!pip install -U -q transformers==4.39.3

In [2]:
import os

os.environ["model_name"]='microsoft/GODEL-v1_1-base-seq2seq'

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer=AutoTokenizer.from_pretrained(os.getenv('model_name'))

tokenizer_config.json:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [4]:
model=AutoModelForSeq2SeqLM.from_pretrained(os.getenv('model_name'), device_map='auto')

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
def generate(instruction, knowledge, dialog):
    if knowledge!='':
        knowledge='[KNOWLEDGE]'+knoledge
    
    dialog='EOS'.join(dialog)
    query=f"{instruction} [CONTEXT] {dialog} {knowledge}"
    return query

instruction=f"Instruction: given a dialog context, you need to response empathically."

knowledge=''

dialog=[
    'How the weather in Melbourne normally?'
]

query=generate(instruction, knowledge, dialog)

input_ids=tokenizer(f"{query}", return_tensors="pt").to('cuda').input_ids

outputs=model.generate(input_ids, max_length=128, min_length=8, top_p=0.9, do_sample=True)
output=tokenizer.decode(outputs[0], skip_special_tokens=True)
output

2024-04-21 05:04:10.635625: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 05:04:10.635725: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 05:04:10.764754: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


'My friend has this thing that he has on the porch of the apartment I live in.'

In [6]:
query

'Instruction: given a dialog context, you need to response empathically. [CONTEXT] How the weather in Melbourne normally? '

In [7]:
# Use a pipeline as a high-level helper
from transformers import pipeline, Conversation

pipe = pipeline(task='text2text-generation',model="microsoft/GODEL-v1_1-base-seq2seq", tokenizer=tokenizer)

In [8]:
pipe(query, max_length=128, min_length=8, top_p=0.9, do_sample=True)

[{'generated_text': 'The average temperature in Melbourne is usually around a constant 0 degree Fahrenheit.'}]

In [9]:
con=Conversation()
con.add_message({"role": "user", "content": "How the weather in Melbourne normally?"})
con.add_message({"role": "assistant", "content": "Melbourne is also sunny which is my favourite weather"})
con.add_message({"role": "user", "content": "why Melbourne is a good place to travel?"})

In [10]:
pipe = pipeline(task='conversational',model="microsoft/GODEL-v1_1-base-seq2seq", tokenizer=tokenizer)

In [11]:
pipe(con, max_length=128, min_length=8, top_p=0.9, do_sample=True)


No chat template is defined for this tokenizer - using a default chat template that implements the ChatML format (without BOS/EOS tokens!). If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



Conversation id: f4cb97e7-f5a7-4980-897e-81204ff69067
user: How the weather in Melbourne normally?
assistant: Melbourne is also sunny which is my favourite weather
user: why Melbourne is a good place to travel?
assistant: i have only ever been there

In [12]:
con[-1].get('content')

'i have only ever been there'

In [13]:
pipe(con)

Conversation id: f4cb97e7-f5a7-4980-897e-81204ff69067
user: How the weather in Melbourne normally?
assistant: Melbourne is also sunny which is my favourite weather
user: why Melbourne is a good place to travel?
assistant: i have only ever been there
assistant: i have only been there|im_end|> |im_start|>assistant i have only been there|im_end|> |im_start|>assistant i have only been there|im_end|> |im_start|>assistant i have only been there|im_end|> |im_start|>assistant i have only been there|im_end|> |im_start|>assistant i have only been there|im_end

In [14]:
pipe(con, max_length=128, min_length=8, top_p=0.9, do_sample=True)

Conversation id: f4cb97e7-f5a7-4980-897e-81204ff69067
user: How the weather in Melbourne normally?
assistant: Melbourne is also sunny which is my favourite weather
user: why Melbourne is a good place to travel?
assistant: i have only ever been there
assistant: i have only been there|im_end|> |im_start|>assistant i have only been there|im_end|> |im_start|>assistant i have only been there|im_end|> |im_start|>assistant i have only been there|im_end|> |im_start|>assistant i have only been there|im_end|> |im_start|>assistant i have only been there|im_end
assistant: Why is Melbourne a good place to travel?

# References

* https://github.com/SkywardAI/chat-backend/issues/108
* https://huggingface.co/microsoft/GODEL-v1_1-base-seq2seq